Runs an extended Kalman filter on Prof Flaxman's SEIIR predictions and
measurement data for New York State from mid-April to 7 July 2020.

S - Susceptible
E - Exposed
I1 - Presymptomatic
I2 - Symptomatic
R - Recovered

In [ ]:
#%load_ext autoreload
#%autoreload

In [ ]:
#%reset

In [1]:
import math
import datetime

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import data_sets
import seiir_compartmental

In [2]:
# functions to support the Kalman filtering
def get_predicts_prior(day, seiir):
    x_hat = np.array([[seiir['S'].loc[day]],
                      [seiir['E'].loc[day]],
                      [seiir['I1'].loc[day]],
                      [seiir['I2'].loc[day]],
                      [seiir['R'].loc[day]]])

    beta_k = seiir['beta_pred'].loc[day]

    return x_hat, beta_k


def step_seiir(x_hat, constants, beta_k, days=7):
    s_dict = {'S': x_hat[0, 0],
              'E': x_hat[1, 0],
              'I1': x_hat[2, 0],
              'I2': x_hat[3, 0],
              'R': x_hat[4, 0]}

    s = pd.Series(s_dict)

    for i in range(days):
        infectious = s.loc['I1'] + s.loc['I2']
        s = seiir_compartmental.compartmental_covid_step(s, s.sum(),
                                                         infectious,
                                                         constants['alpha'],
                                                         beta_k,
                                                         constants['gamma1'],
                                                         constants['gamma2'],
                                                         constants['sigma'],
                                                         constants['theta'])
    x_hat_future_prior = np.array([[s.loc['S']],
                                   [s.loc['E']],
                                   [s.loc['I1']],
                                   [s.loc['I2']],
                                   [s.loc['R']]])

    return x_hat_future_prior


def predict_step(x_hat_k1_prior, P, Q, beta_k, constants):
    S = x_hat_k1_prior[0, 0]
    E = x_hat_k1_prior[1, 0]
    I1 = x_hat_k1_prior[2, 0]
    I2 = x_hat_k1_prior[3, 0]
    R = x_hat_k1_prior[4, 0]
    N = S + E + I1 + I2 + R
    alpha = constants['alpha']
    sigma = constants['sigma']
    gamma1 = constants['gamma1']
    gamma2 = constants['gamma2']

    part_f_S = np.array([[-beta_k * math.pow(I1 + I2, alpha) / N],
                         [beta_k * math.pow(I1 + I2, alpha) / N],
                         [0],
                         [0],
                         [0]])

    part_f_E = np.array([[0],
                         [-sigma],
                         [sigma],
                         [0],
                         [0]])

    part_f_I1 = np.array([[-alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [-gamma1],
                          [gamma1],
                          [0]])

    part_f_I2 = np.array([[-alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [0],
                          [-gamma2],
                          [gamma2]])

    part_f_R = np.array([[0],
                         [0],
                         [0],
                         [0],
                         [0]])

    # 5x5
    f_jacob = np.concatenate([part_f_S, part_f_E, part_f_I1, part_f_I2,
                              part_f_R], axis=1)

    # 5x5
    # P_k1_prior = f_jacob * P * f_jacob^T + Q
    P_k1_prior = np.matmul(np.matmul(f_jacob, P), np.transpose(f_jacob)) + Q
    return P_k1_prior


def update_step(x_hat, x_hat_k1, P_k1, Rn, rho1, rho2, z_k):
    # 5x5
    ep = 10**-10
    H = np.array([[ep, 0, 0, 0, 0],
                  [0, ep, 0, 0, 0],
                  [0, 0, ep, rho1, 0],
                  [0, 0, 0, rho2, 0],
                  [0, 0, 0, 0, ep]])
    #H = np.array([[0, 0, 0, 0, 0],
    #                  [0, 0, 0, 0, 0],
    #                  [0, 0, 0, rho1, 0],
    #                  [0, 0, 0, rho2, 0],
    #                  [0, 0, 0, 0, 0]])
    
    # Si = H * P_k1 * H^T + Rn
    Si = np.matmul(np.matmul(H, P_k1), np.transpose(H)) + Rn

    # K_new = P_k1 * H^T * Si^(-1)
    K_new = np.matmul(np.matmul(P_k1, np.transpose(H)), np.linalg.inv(Si))
    y_new = np.matmul(H, x_hat)

    # 5x1
    diff = z_k - y_new

    x_hat_k1_post = x_hat_k1 + np.matmul(K_new, diff)

    P_k1_post = P_k1 - np.matmul(np.matmul(K_new, Si), np.transpose(K_new))

    # joseph formulation
    #joe2 = np.matmul(np.matmul(K_new, Rn), np.transpose(K_new))
    #joe1 = np.eye(5) - np.matmul(K_new, H)
    #P_k1_post = np.matmul(np.matmul(joe1, P_k1), np.transpose(joe1)) - joe2

    return x_hat_k1_post, P_k1_post

def get_predict_measures():
    predictions = pd.read_csv(
        r'case_vs_symptom/kalman/predicted_measures/predictions.csv',
        header=None, names=['measure_date', 'prediction_date', 'prop', 'case',
                            'prop_pred7', 'case_pred7', 'prop_pred1',
                            'case_pred1'],
        index_col=False, usecols=[0, 4, 5])

    start_d = datetime.timedelta(days=int(predictions['measure_date'].iloc[0]))
    start = datetime.date(2019, 12, 31) + start_d
    end_d = datetime.timedelta(days=int(predictions['measure_date'].iloc[-1]))
    end = datetime.date(2019, 12, 31) + end_d

    measure_rng = pd.date_range(start=start, end=end)

    # code to save:
    # prediction_rng = pd.date_range(start=datetime.date(2019, 12, 31) +
    # datetime.timedelta(days=int(predictions['prediction_date'].iloc[0])),
    # end=datetime.date(2019, 12, 31) +
    # datetime.timedelta(days=int(predictions['prediction_date'].iloc[-1])))

    predictions.set_index(measure_rng, inplace=True)

    return predictions


def get_data_sets(state, fips=None):
    # the post hoc seiir model predictions provided by Prof Flaxman without
    # any kalman filtering:
    #seiir = pd.read_csv(r'data/seiir_compartments_post-hoc_ny_state_20200910.csv', header=0,
    #                    index_col='date', parse_dates=True)
    seiir = pd.read_csv(r'data/seiir_compartments_post-hoc_ny_state.csv', header=0,
                        index_col='date', parse_dates=True)
    

    fb_data = data_sets.create_symptom_df()
    fb_data_val = data_sets.create_symptom_df(valid=True)

    if fips is None:
        case_data = data_sets.create_case_df_state()
        case_data_geo = case_data.loc[state]['case_rate'].copy()
        fb_data_geo = fb_data.loc[state].groupby('date').sum().copy()
        fb_data_val_geo = fb_data_val.loc[state].groupby('date').sum().copy()

    else:
        case_data = data_sets.create_case_df_county()
        case_data_geo = case_data.loc[fips]['case_rate'].copy()
        
        if fips == 'New York City':
            nyc_fips = ['36005', '36061', '36047', '36085']
            fb_data_geo = fb_data.loc[(slice(None), '36081'), :].copy()
            fb_data_geo = fb_data_geo.mean(level='date')
            fb_data_val_geo = fb_data_val.loc[(slice(None), '36081'), :].copy()
            fb_data_val_geo = fb_data_val.mean(level='date')
            for borough in nyc_fips:
                fb_data_geo += fb_data.loc[(slice(None), borough), :].copy().mean(level='date')
                fb_data_val_geo += fb_data_val.loc[(slice(None), borough), :].copy().mean(level='date')
        else:
            fb_data_geo = fb_data.loc[(slice(None), fips), :].copy()
            fb_data_val_geo = fb_data_val.loc[(slice(None), fips), :].copy()
            # collapse down to a single index column (date)
            fb_data_geo = fb_data_geo.mean(level='date')
            fb_data_val_geo = fb_data_val_geo.mean(level='date')


    return seiir, fb_data_geo, fb_data_val_geo, case_data_geo


def calc_fb_ma7(fb_data):
    """
    Returns a Pandas series
    """
    # the fb_data is a DataFrame while the case_data is a Series
    fb_ma7 = fb_data.rolling(window=7).mean()
    fb_ma7 = fb_ma7.iloc[6:, :]
    prop_ma7 = fb_ma7['num_stl'].div(fb_ma7['n'])

    return prop_ma7, fb_ma7['n'].copy(), fb_ma7['num_stl'].copy()

In [3]:
# set constants
K0 = datetime.date(2020, 4, 12)

the_state = 'NY'
the_county = data_sets.get_fips(the_state, 'Albany')
#the_county = 'New York City'
constants = {
    'alpha': 0.948786,
    'gamma1': 0.500000,
    'gamma2': 0.662215,
    'sigma': 0.266635,
    'theta': 6.000000
    }

# set initial values for Kalman filter parameters
P_mult = 1
Q_mult = 1

# Rn is the R noise matrix; it remains constant thru the stepping of the
# Kalman filter
# prior to experiments, Rn_mult = 5*10**-4
Rn_mult = 5*10**-8

Rn_22 = 88
Rn_32 = 150

Rn_23 = 0
Rn_33 = 1

Rn = Rn_mult * np.array([[0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0],
                         [0, 0, Rn_22, 0, 0],
                         [0, 0, Rn_32, Rn_33, 0],
                         [0, 0, 0, 0, 0]])
Q = Q_mult * np.eye(5)
P = P_mult * np.eye(5)

# generate data
seiir, fb_data, fb_data_val, case_data = get_data_sets(
    data_sets.STATES[the_state], fips=the_county)


if the_county == 'New York City':
    county_pop = 0
    for each in ['36081', '36005', '36061', '36047', '36085']:
        this_count, state_pop = data_sets.get_pops(each)
        county_pop += this_count
else:
    county_pop, state_pop = data_sets.get_pops(the_county)

b = county_pop / state_pop

# calculate moving averages on the fb and case data
case_ma7 = case_data.rolling(window=7).mean()
case_ma7_all = case_ma7.iloc[6:]
prop_ma7, n_ma7, num_stl_ma7 = calc_fb_ma7(fb_data)
prop_ma7_valid, n_ma7_valid, num_stl_ma7_valid = calc_fb_ma7(fb_data_val)

# get starting compartment values for the state level
x_hat_state_k0, beta_k0 = get_predicts_prior(K0, seiir)
x_hat_k0 = b * x_hat_state_k0
I2_county = x_hat_k0[3, 0]
rho1 = prop_ma7.loc['2020-04-12'] / I2_county
rho2 = case_ma7_all.loc['2020-04-12'] / I2_county


# approximate rho values
# I2_county = b * I2
# prop_county = rho1 * I2_county
# case_county = rho2 * I2_county


# create empty dictionaries to hold the estimated values
prop_est = {}
case_est = {}
seiir_pred = {}



# Original data run ----------------
start = K0
d = start
while d <= datetime.date(2020, 9, 30):
    # each cycle of the while loop executes a step

    # get state level compartments
    x_hat_state_k, beta_k = get_predicts_prior(d, seiir)

    # step the state level compartments 7 days forward
    x_hat_state_k1 = step_seiir(x_hat_state_k, constants, beta_k)

    # convert the state level compartments to county level values
    x_hat_k = b * x_hat_state_k
    x_hat_k1 = b * x_hat_state_k1

    # get measurements
    z_k = np.array([[0],
                    [0],
                    [prop_ma7.loc[d]],
                    [case_ma7_all.loc[d]],
                    [0]])

    # predict step
    P = predict_step(x_hat_k1, P, Q, beta_k, constants)
    print('step -----------')
    print('P:')
    print(P)

    # update step
    x_hat_post, P_post = update_step(x_hat_k, x_hat_k1, P, Rn,
                                     rho1, rho2, z_k)
    print('P_post:')
    print(P_post)

    # store estimated values for proportion and case rate
    indexDate = d + datetime.timedelta(days=7)
    prop_est[indexDate] = rho1 * x_hat_post[3, 0]
    case_est[indexDate] = rho2 * x_hat_post[3, 0]
    seiir_pred[indexDate] = b * x_hat_k1[3, 0]

    # update the P and d
    P = P_post
    d += datetime.timedelta(days=1)
    
# create pandas series of the estimated case rate
predicted_case = pd.Series(case_est)
predicted_seiir_prior = pd.Series(seiir_pred)

step -----------
P:
[[ 1.15265515 -0.15265515  0.1381319   0.04481413 -0.18294603]
 [-0.15265515  1.22374937 -0.20922612 -0.04481413  0.18294603]
 [ 0.1381319  -0.20922612  1.32109422 -0.25        0.        ]
 [ 0.04481413 -0.04481413 -0.25        1.68852871 -0.43852871]
 [-0.18294603  0.18294603  0.         -0.43852871  1.43852871]]
P_post:
[[ 4.44089210e-16  1.66533454e-16  4.99600361e-16 -3.51801921e-15
   1.02695630e-15]
 [ 1.66533454e-16  2.22044605e-16  8.32667268e-17 -8.25728375e-16
   2.77555756e-16]
 [ 4.99600361e-16  8.32667268e-17  1.23339254e+00 -4.44724446e-05
   6.42691708e-17]
 [-3.54577478e-15 -8.46545056e-16 -4.44881817e-05  2.79928156e-04
   1.11022302e-16]
 [ 1.08246745e-15  3.05311332e-16  8.18087319e-17  0.00000000e+00
  -4.44089210e-16]]
step -----------
P:
[[ 1.09292690e+00 -9.29268977e-02  1.69255230e-01 -1.69212435e-01
  -4.27951631e-05]
 [-9.29268977e-02  1.09292690e+00 -1.69255230e-01  1.69212435e-01
   4.27951631e-05]
 [ 1.69255233e-01 -1.69255233e-01  1.308

step -----------
P:
[[ 1.08595338e+00 -8.59533847e-02  1.60853979e-01 -1.60814579e-01
  -3.93997747e-05]
 [-8.59533846e-02  1.08595338e+00 -1.60853979e-01  1.60814579e-01
   3.93997750e-05]
 [ 1.60853982e-01 -1.60853982e-01  1.30109363e+00 -3.01112570e-01
   1.89356123e-05]
 [-1.60814586e-01  1.60814586e-01 -3.01112577e-01  1.30125426e+00
  -1.41686912e-04]
 [-3.93960425e-05  3.93960425e-05  1.89425984e-05 -1.41693897e-04
   1.00012275e+00]]
P_post:
[[-2.22044605e-16  8.32667268e-17 -1.38399470e-09  1.46260781e-12
   3.38813179e-20]
 [ 9.71445147e-17  2.22044605e-16  1.52176294e-09 -1.50071622e-12
   1.35525272e-20]
 [-2.22044605e-16  0.00000000e+00  1.20442203e+00 -5.72020044e-05
   3.38813179e-20]
 [ 6.66133815e-16 -1.11022302e-16 -5.72231073e-05  2.79916226e-04
  -1.62630326e-19]
 [ 4.06575815e-20  6.77626358e-21  4.74318295e-09 -1.32735118e-12
   0.00000000e+00]]
step -----------
P:
[[ 1.08597189e+00 -8.59718873e-02  1.60874463e-01 -1.60835062e-01
  -3.94008885e-05]
 [-8.59718872e-

step -----------
P:
[[ 1.08660776e+00 -8.66077573e-02  1.61456885e-01 -1.61417327e-01
  -3.95575801e-05]
 [-8.66077572e-02  1.08660776e+00 -1.61456885e-01  1.61417327e-01
   3.95575804e-05]
 [ 1.61456888e-01 -1.61456888e-01  1.30106294e+00 -3.01081864e-01
   1.89242396e-05]
 [-1.61417334e-01  1.61417334e-01 -3.01081871e-01  1.30122355e+00
  -1.41675535e-04]
 [-3.95538346e-05  3.95538346e-05  1.89312236e-05 -1.41682518e-04
   1.00012275e+00]]
P_post:
[[ 6.66133815e-16 -1.94289029e-16 -1.38947709e-09  1.46790913e-12
  -1.82959117e-19]
 [-1.94289029e-16  4.44089210e-16  1.52753885e-09 -1.50621182e-12
  -7.45388994e-20]
 [-2.77555756e-17 -2.77555756e-17  1.20422401e+00 -5.71466026e-05
  -1.35525272e-20]
 [ 4.99600361e-16 -2.77555756e-17 -5.71676923e-05  2.79916211e-04
   2.71050543e-20]
 [-1.82959117e-19 -7.45388994e-20  4.73768569e-09 -1.32581280e-12
  -2.22044605e-16]]
step -----------
P:
[[ 1.08650615e+00 -8.65061477e-02  1.61360285e-01 -1.61320749e-01
  -3.95362015e-05]
 [-8.65061476e-

step -----------
P:
[[ 1.08758927e+00 -8.75892681e-02  1.62353439e-01 -1.62313643e-01
  -3.97966210e-05]
 [-8.75892680e-02  1.08758927e+00 -1.62353439e-01  1.62313642e-01
   3.97966213e-05]
 [ 1.62353442e-01 -1.62353442e-01  1.30100455e+00 -3.01023452e-01
   1.89026034e-05]
 [-1.62313649e-01  1.62313649e-01 -3.01023459e-01  1.30116511e+00
  -1.41653890e-04]
 [-3.97928571e-05  3.97928571e-05  1.89095817e-05 -1.41660868e-04
   1.00012275e+00]]
P_post:
[[-2.22044605e-16 -6.93889390e-17 -1.39760264e-09  1.47445944e-12
   7.45388994e-20]
 [-6.93889390e-17  2.22044605e-16  1.53607338e-09 -1.51390012e-12
  -1.35525272e-20]
 [ 1.66533454e-16 -5.55111512e-17  1.20392097e+00 -5.70618193e-05
   1.35525272e-20]
 [-6.66133815e-16  1.11022302e-16 -5.70828870e-05  2.79916187e-04
  -5.42101086e-20]
 [ 7.45388994e-20 -6.77626358e-21  4.72768334e-09 -1.32301399e-12
   2.22044605e-16]]
step -----------
P:
[[ 1.08830167e+00 -8.83016676e-02  1.63005762e-01 -1.62965798e-01
  -3.99646238e-05]
 [-8.83016675e-

step -----------
P:
[[ 1.09143212e+00 -9.14321201e-02  1.65780237e-01 -1.65739482e-01
  -4.07553944e-05]
 [-9.14321200e-02  1.09143212e+00 -1.65780237e-01  1.65739482e-01
   4.07553947e-05]
 [ 1.65780240e-01 -1.65780240e-01  1.30065457e+00 -3.00673344e-01
   1.87729155e-05]
 [-1.65739488e-01  1.65739488e-01 -3.00673351e-01  1.30081488e+00
  -1.41524154e-04]
 [-4.07515659e-05  4.07515659e-05  1.87798588e-05 -1.41531097e-04
   1.00012275e+00]]
P_post:
[[ 6.66133815e-16 -1.38777878e-16 -1.42709317e-09  1.50374158e-12
  -2.43945489e-19]
 [-1.38777878e-16 -4.44089210e-16  1.56824595e-09 -1.54254387e-12
   6.77626358e-21]
 [-2.77555756e-17  2.77555756e-17  1.20267679e+00 -5.67137075e-05
  -3.72694497e-20]
 [ 8.60422844e-16  2.77555756e-17 -5.67346797e-05  2.79916090e-04
   2.71050543e-20]
 [-2.43945489e-19  0.00000000e+00  4.68070032e-09 -1.30986639e-12
   0.00000000e+00]]
step -----------
P:
[[ 1.09112144e+00 -9.11214379e-02  1.65502367e-01 -1.65461685e-01
  -4.06821225e-05]
 [-9.11214378e-

...

step -----------
P:
[[ 1.12053867e+00 -1.20538669e-01  1.89680601e-01 -1.89633147e-01
  -4.74539285e-05]
 [-1.20538669e-01  1.12053867e+00 -1.89680600e-01  1.89633147e-01
   4.74539288e-05]
 [ 1.89680604e-01 -1.89680604e-01  1.29855286e+00 -2.98570858e-01
   1.79941150e-05]
 [-1.89633154e-01  1.89633154e-01 -2.98570865e-01  1.29871161e+00
  -1.40745065e-04]
 [-4.74496471e-05  4.74496471e-05  1.80008538e-05 -1.40751804e-04
   1.00012275e+00]]
P_post:
[[-2.22044605e-16  2.08166817e-16 -1.64292735e-09  1.70072290e-12
  -1.62630326e-19]
 [ 2.08166817e-16  8.88178420e-16  1.79808929e-09 -1.74188441e-12
  -1.62630326e-19]
 [-1.94289029e-16 -4.16333634e-16  1.19392587e+00 -5.42653294e-05
  -3.38813179e-21]
 [ 4.16333634e-16  1.88737914e-15 -5.42856593e-05  2.79915405e-04
  -2.71050543e-20]
 [-1.55854062e-19 -1.62630326e-19  4.37730857e-09 -1.22496932e-12
   0.00000000e+00]]
step -----------
P:
[[ 1.12227979e+00 -1.22279792e-01  1.91022761e-01 -1.90974943e-01
  -4.78180647e-05]
 [-1.22279792e-

step -----------
P:
[[ 1.13106278e+00 -1.31062775e-01  1.97518045e-01 -1.97468296e-01
  -4.97495954e-05]
 [-1.31062775e-01  1.13106278e+00 -1.97518045e-01  1.97468295e-01
   4.97495957e-05]
 [ 1.97518048e-01 -1.97518048e-01  1.29773938e+00 -2.97757070e-01
   1.76926682e-05]
 [-1.97468303e-01  1.97468303e-01 -2.97757077e-01  1.29789752e+00
  -1.40443507e-04]
 [-4.97451774e-05  4.97451774e-05  1.76993279e-05 -1.40450166e-04
   1.00012275e+00]]
P_post:
[[ 4.44089210e-16 -3.05311332e-16 -1.71187470e-09  1.76314519e-12
  -1.89735380e-19]
 [-3.05311332e-16 -2.22044605e-16  1.87158486e-09 -1.80633286e-12
   0.00000000e+00]
 [-2.49800181e-16  0.00000000e+00  1.19088901e+00 -5.34156552e-05
  -1.01643954e-20]
 [ 1.58206781e-15 -2.77555756e-17 -5.34357633e-05  2.79915167e-04
  -2.71050543e-20]
 [-1.82959117e-19  6.77626358e-21  4.26985693e-09 -1.19490130e-12
  -2.22044605e-16]]
step -----------
P:
[[ 1.13152675e+00 -1.31526746e-01  1.97861658e-01 -1.97811815e-01
  -4.98432238e-05]
 [-1.31526746e-

step -----------
P:
[[ 1.12945135e+00 -1.29451348e-01  1.96327762e-01 -1.96278347e-01
  -4.94153036e-05]
 [-1.29451348e-01  1.12945135e+00 -1.96327762e-01  1.96278346e-01
   4.94153039e-05]
 [ 1.96327765e-01 -1.96327765e-01  1.29782346e+00 -2.97841182e-01
   1.77238241e-05]
 [-1.96278354e-01  1.96278354e-01 -2.97841189e-01  1.29798166e+00
  -1.40474674e-04]
 [-4.94109086e-05  4.94109086e-05  1.77304912e-05 -1.40481341e-04
   1.00012275e+00]]
P_post:
[[-2.22044605e-16  2.49800181e-16 -1.70037517e-09  1.75184867e-12
  -2.03287907e-19]
 [ 2.49800181e-16  2.22044605e-16  1.85959884e-09 -1.79603554e-12
   3.38813179e-20]
 [-2.22044605e-16  5.55111512e-17  1.19132786e+00 -5.35384364e-05
  -4.40457133e-20]
 [ 5.27355937e-16 -2.77555756e-17 -5.35585745e-05  2.79915201e-04
   1.08420217e-19]
 [-1.96511644e-19  2.03287907e-20  4.28319028e-09 -1.19863212e-12
  -2.22044605e-16]]
step -----------
P:
[[ 1.12904659e+00 -1.29046593e-01  1.96023393e-01 -1.95974058e-01
  -4.93352107e-05]
 [-1.29046593e-

step -----------
P:
[[ 1.13333305e+00 -1.33333054e-01  1.99175135e-01 -1.99124910e-01
  -5.02245586e-05]
 [-1.33333054e-01  1.13333305e+00 -1.99175135e-01  1.99124910e-01
   5.02245589e-05]
 [ 1.99175138e-01 -1.99175138e-01  1.29760110e+00 -2.97618745e-01
   1.76414323e-05]
 [-1.99124918e-01  1.99124918e-01 -2.97618752e-01  1.29775914e+00
  -1.40392252e-04]
 [-5.02201098e-05  5.02201098e-05  1.76480795e-05 -1.40398899e-04
   1.00012275e+00]]
P_post:
[[ 6.66133815e-16 -2.49800181e-16 -1.72730241e-09  1.77702297e-12
  -7.45388994e-20]
 [-2.77555756e-16 -2.22044605e-16  1.88789992e-09 -1.82029392e-12
  -2.03287907e-20]
 [-1.94289029e-16  0.00000000e+00  1.19026346e+00 -5.32406382e-05
  -3.38813179e-21]
 [ 1.83186799e-15 -2.77555756e-17 -5.32607028e-05  2.79915118e-04
  -5.42101086e-20]
 [-5.42101086e-20 -2.03287907e-20  4.24981043e-09 -1.18929188e-12
  -4.44089210e-16]]
step -----------
P:
[[ 1.13325948e+00 -1.33259481e-01  1.99108383e-01 -1.99058160e-01
  -5.02224059e-05]
 [-1.33259480e-

P_post:
[[-4.44089210e-16 -3.33066907e-16 -1.78453108e-09  1.82467930e-12
   6.09863722e-20]
 [-3.60822483e-16  2.22044605e-16  1.94894251e-09 -1.87183602e-12
   0.00000000e+00]
 [ 3.05311332e-16 -1.11022302e-16  1.18779158e+00 -5.25490451e-05
  -2.03287907e-20]
 [-1.36002321e-15  1.94289029e-16 -5.25689358e-05  2.79914924e-04
   1.35525272e-19]
 [ 6.09863722e-20  6.77626358e-21  4.16731105e-09 -1.16620599e-12
   2.22044605e-16]]
step -----------
P:
[[ 1.14666456e+00 -1.46664562e-01  2.08665930e-01 -2.08613027e-01
  -5.29035511e-05]
 [-1.46664562e-01  1.14666456e+00 -2.08665930e-01  2.08613026e-01
   5.29035514e-05]
 [ 2.08665933e-01 -2.08665933e-01  1.29694789e+00 -2.96965294e-01
   1.73993826e-05]
 [-2.08613035e-01  2.08613035e-01 -2.96965301e-01  1.29710545e+00
  -1.40150113e-04]
 [-5.28989229e-05  5.28989229e-05  1.74059689e-05 -1.40156698e-04
   1.00012275e+00]]
P_post:
[[ 8.88178420e-16 -3.60822483e-16 -1.81959658e-09  1.85476634e-12
   6.77626358e-20]
 [-4.16333634e-16  4.440892

In [32]:
# compute squared error for the overlapping time period -------------------------

left = predicted_seiir_prior.index[0]
right = d

seiir_err = case_ma7_all.loc[left:right] - predicted_seiir_prior[left:right]
seiir_sum_sq_err = (seiir_err**2).sum()
seiir_mse = seiir_sum_sq_err / seiir_err.count()

kalman_err = predicted_case.loc[left:right] - case_ma7_all.loc[left:right]
kalman_sum_sq_err = (kalman_err**2).sum()
kalman_mse = kalman_sum_sq_err / kalman_err.count()


print('SSE between seiir forecast and case rate:', seiir_sum_sq_err)
print('MSE between seiir forecast and measured case rate:', seiir_mse)
print()
print('SSE between kalman forecast and case rate:', kalman_sum_sq_err)
print('MSE between kalman forecast and measured case rate:', kalman_mse)

SSE between seiir forecast and case rate: 35011.983114309725
MSE between seiir forecast and measured case rate: 210.91556092957666

SSE between kalman forecast and case rate: 11190.423881387616
MSE between kalman forecast and measured case rate: 68.65290724777678


In [36]:
predicted_case.tail(40)

2020-08-29     7.203854
2020-08-30     6.501437
2020-08-31     6.361182
2020-09-01     7.211996
2020-09-02     6.237179
2020-09-03     7.512750
2020-09-04     7.943670
2020-09-05     8.799505
2020-09-06     9.373789
2020-09-07     9.945258
2020-09-08          NaN
2020-09-09          NaN
2020-09-10          NaN
2020-09-11    10.552629
2020-09-12     9.979768
2020-09-13     9.696799
2020-09-14     9.114620
2020-09-15     7.830105
2020-09-16     7.988045
2020-09-17     8.282781
2020-09-18     9.123947
2020-09-19    10.543206
2020-09-20    12.118691
2020-09-21    12.689423
2020-09-22    13.841051
2020-09-23    14.691538
2020-09-24    15.817370
2020-09-25    13.973322
2020-09-26    15.851120
2020-09-27    14.527312
2020-09-28    14.826221
2020-09-29    14.526508
2020-09-30    13.678380
2020-10-01    12.096023
2020-10-02    12.129732
2020-10-03    10.950990
2020-10-04    11.402483
2020-10-05    11.547491
2020-10-06    11.412099
2020-10-07    11.150627
dtype: float64

In [31]:
len(seiir_err)

166

In [23]:
predicted_case.tail(40)


2020-08-29     7.203854
2020-08-30     6.501437
2020-08-31     6.361182
2020-09-01     7.211996
2020-09-02     6.237179
2020-09-03     7.512750
2020-09-04     7.943670
2020-09-05     8.799505
2020-09-06     9.373789
2020-09-07     9.945258
2020-09-08          NaN
2020-09-09          NaN
2020-09-10          NaN
2020-09-11    10.552629
2020-09-12     9.979768
2020-09-13     9.696799
2020-09-14     9.114620
2020-09-15     7.830105
2020-09-16     7.988045
2020-09-17     8.282781
2020-09-18     9.123947
2020-09-19    10.543206
2020-09-20    12.118691
2020-09-21    12.689423
2020-09-22    13.841051
2020-09-23    14.691538
2020-09-24    15.817370
2020-09-25    13.973322
2020-09-26    15.851120
2020-09-27    14.527312
2020-09-28    14.826221
2020-09-29    14.526508
2020-09-30    13.678380
2020-10-01    12.096023
2020-10-02    12.129732
2020-10-03    10.950990
2020-10-04    11.402483
2020-10-05    11.547491
2020-10-06    11.412099
2020-10-07    11.150627
dtype: float64

In [17]:
d

datetime.date(2020, 10, 1)

In [14]:
inner.tail(40)

date
2020-05-29     61.992099
2020-05-30     52.237484
2020-05-31     65.310163
2020-06-01    101.280563
2020-06-02    153.863683
2020-06-03    222.508070
2020-06-04    192.615132
2020-06-05    160.435168
2020-06-06    174.550608
2020-06-07    144.972377
2020-06-08    130.674438
2020-06-09     92.474806
2020-06-10     53.013991
2020-06-11     54.255235
2020-06-12     55.536354
2020-06-13     50.538631
2020-06-14     44.120322
2020-06-15     20.950705
2020-06-16     19.906421
2020-06-17     20.724178
2020-06-18     15.395176
2020-06-19     20.271939
2020-06-20     14.687890
2020-06-21     11.927088
2020-06-22     16.710616
2020-06-23     14.674279
2020-06-24     14.837404
2020-06-25     22.532219
2020-06-26     14.544975
2020-06-27     18.016089
2020-06-28     24.440366
2020-06-29     19.540511
2020-06-30     21.250243
2020-07-01     31.727766
2020-07-02     35.349803
2020-07-03     30.560105
2020-07-04     44.981463
2020-07-05     46.869378
2020-07-06     59.689081
2020-07-07     62.24

In [ ]:
# MASE calculation -----------------

left = predicted_case.index[0]
right = predicted_case.index[-1]

# take error between prediction and actual
e = (case_ma7_all.loc[left:right] - predicted_case.loc[left:right]).abs()

case_ma7_diff = case_ma7_all.diff(7)
denom = (case_ma7_diff.loc[left:right]).abs()

term = e.div(denom)
term.replace([np.inf, -np.inf], np.nan, inplace=True)
term.dropna(inplace=True)
mase = term.sum() / len(term)
print('mase:', mase)

In [ ]:
# Not currently used ---------------
# Validation data run ---------------------
start = prop_ma7_valid.index[0]
d = start
while d <= prop_ma7_valid.index[-1]:
    # each cycle of the while loop executes a step

    # get state level compartments
    x_hat_state_k, beta_k = get_predicts_prior(d, seiir)

    # step the state level compartments 7 days forward
    x_hat_state_k1 = step_seiir(x_hat_state_k, constants, beta_k)

    # convert the state level compartments to county level values
    x_hat_k = b * x_hat_state_k
    x_hat_k1 = b * x_hat_state_k1

    # get measurements
    z_k = np.array([[0],
                    [0],
                    [prop_ma7_valid.loc[d]],
                    [case_ma7_all.loc[d]],
                    [0]])

    # predict step
    P = predict_step(x_hat_k1, P, Q, beta_k, constants)
    print('step -----------')
    print('P:')
    print(P)

    # update step
    x_hat_post, P_post = update_step(x_hat_k, x_hat_k1, P, Rn,
                                     rho1, rho2, z_k)
    print('P_post:')
    print(P_post)

    # store estimated values for proportion and case rate
    indexDate = d + datetime.timedelta(days=7)
    prop_est[indexDate] = rho1 * x_hat_post[3, 0]
    case_est[indexDate] = rho2 * x_hat_post[3, 0]
    seiir_pred[indexDate] = b * x_hat_k1[3, 0]

    # update the P and d
    P = P_post
    d += datetime.timedelta(days=1)
    
# create pandas series of the estimated case rate
predicted_case = pd.Series(case_est)
predicted_seiir_prior = pd.Series(seiir_pred)

In [ ]:
# Plotting constants and variables ----------------
matplotlib.rcParams.update({'font.size': 20})
plt.style.use('seaborn-whitegrid')
purple = '#33016F'
gold = '#9E7A27'
gray = '#797979'
width = 4
%matplotlib qt

tick_end = predicted_seiir_prior.index[-1]

week_interval = pd.date_range(start=start, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]

In [ ]:
# Single plot with all lines

matplotlib.rcParams.update({'font.size': 20})
plt.style.use('seaborn-whitegrid')
tick_start = K0
tick_end = predicted_seiir_prior.index[-1]



fig3, ax31 = plt.subplots(1)
plt.sca(ax31)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Case Positive Rate', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.plot(predicted_case.index, predicted_case, label='Our 7-Day Forecast',
         c=purple, linewidth=width)
#plt.ylim(-2, 72)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')
# plt.xlabel('Date')
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')

ax32 = ax31.twinx()
plt.sca(ax32)
plt.plot(prop_ma7.index, num_stl_ma7, c='red', label='Facebook Positive Symptoms',
         linewidth=width)
#plt.ylim(-.065, 2.5)
plt.grid(axis='y', linestyle=':')
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')

plt.show()

In [ ]:
# plot validation data

fig_v, ax_v = plt.subplots(1)
plt.sca(ax_v)


plt.plot(case_ma7_all.loc[start:end].index, case_ma7_all.loc[start:end], label='Case Positive Rate', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.plot(predicted_case.index, predicted_case, label='Our 7-Day Forecast',
         c=purple, linewidth=width)
#plt.ylim(-2, 72)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')
# plt.xlabel('Date')
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')

ax32 = ax_v.twinx()
plt.sca(ax32)
plt.plot(prop_ma7_valid.index, 100*prop_ma7_valid, c='red', label='Facebook Symptom Rate',
         linewidth=width)
plt.ylim(-.065, 2.5)
plt.grid(axis='y', linestyle=':')
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')

plt.ylabel('Percentage of Positive Symptom Response')
plt.legend(loc='upper right')

plt.show()

In [ ]:
# plot findings -- multiple plots
matplotlib.rcParams.update({'font.size': 20})
plt.style.use('seaborn-whitegrid')
tick_start = K0
tick_end = predicted_seiir_prior.index[-1]

week_interval = pd.date_range(start=tick_start, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]

fig1, ax11 = plt.subplots(1)
plt.sca(ax11)
width = 4
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Case Positive Rate', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.ylim(-2, 72)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')
plt.ylabel('Number of Cases per Day')
# plt.xlabel('Date')
plt.legend(loc='upper left')

fig2, ax21 = plt.subplots(1)
plt.sca(ax21)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Case Positive Rate', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.plot(predicted_case.index, predicted_case, label='Our 7-Day Forecast',
         c=purple, linewidth=width)
plt.ylim(-2, 72)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')
# plt.xlabel('Date')
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')

fig3, ax31 = plt.subplots(1)
plt.sca(ax31)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Case Positive Rate', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.plot(predicted_case.index, predicted_case, label='Our 7-Day Forecast',
         c=purple, linewidth=width)
plt.ylim(-2, 72)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')
# plt.xlabel('Date')
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')

ax32 = ax31.twinx()
plt.sca(ax32)
plt.plot(prop_ma7.index, 100*prop_ma7, c='red', label='Facebook Symptom Rate',
         linewidth=width)
plt.ylim(-.065, 2.5)
plt.grid(axis='y', linestyle=':')
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')

plt.ylabel('Percentage of Positive Symptom Response')
plt.legend(loc='upper right')


plt.show()